### Data

Go to: https://data.bts.gov/Research-and-Statistics/Trips-by-Distance/w96p-f2qv
        
- Top Right Corner, Select: `Export`
- Select `CSV`

In [1]:
# ALL IMPORTS
import os
import pandas as pd
import numpy as np 

pd.set_option('display.max_rows', 500)

In [2]:
# For FIPS sans leading zero
def add_zero(x):
    if len(x)<5:
        return "0" + x
    else:
        return x

In [3]:
#timestamp
def stamper(x):
    #temp = x['Date'].split("/")
    temp = x.split("/")

    m = temp[0]
    d = temp[1]
    y = temp[2]
    
    if len(m) == 1:
        m = "0" + m

    if len(d) == 1:
        d = "0" + d
     
    y = "20" + y
    
    return f'{y}-{m}-{d}'

In [4]:
# Read in data
trips_fn = "Trips_by_Distance.csv"
wrkdir = os.getcwd()

df_full = pd.read_csv(f"{wrkdir}/{trips_fn}", sep=",", converters={'County FIPS': lambda x: add_zero(str(x))},engine='python')

In [5]:
# Add timestamp
df = df_full.copy()
df["Timestamp"] = df.Date.apply(lambda x: stamper(x))

### Format

In [6]:
# Delete unneeded columns
delete_me = ["Level", "Date", "State FIPS"]
for me in delete_me:
    del df[me]
    
# Rename column
df.rename(columns={"State Postal Code": "State"}, inplace=True)

# Drop "County" from County Name
df["County Name"]=  df["County Name"].apply(lambda x: str(x).replace("County", "").strip())

# Reorder columns
new_cols = ['Timestamp', 'County Name', 'State', 'County FIPS','Population Staying at Home',
            'Population Not Staying at Home', 'Number of Trips',
            'Number of Trips <1', 'Number of Trips 1-3', 'Number of Trips 3-5',
            'Number of Trips 5-10', 'Number of Trips 10-25',
            'Number of Trips 25-50', 'Number of Trips 50-100',
            'Number of Trips 100-250', 'Number of Trips 250-500',
            'Number of Trips >=500']
df = df[new_cols]

# Delete rows without data
df = df.dropna()

# Convert people to integers
cols = list(df.columns[4:])
for col in cols:
    df[col]= df[col].astype(int)

In [7]:
# Write to csv
df.to_csv(fr'{wrkdir}/county_mobility.csv', index = False)